In [ ]:
import findspark
findspark.init('/opt/spark') 
import pandas as pd
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [ ]:
df = spark.read.csv("output.csv", header=True) 

cluster = Cluster(['localhost'])
session = cluster.connect()

keyspace_name = 'tam8'
table_name = 'my_table8'

query_create_keyspace = f"CREATE KEYSPACE IF NOT EXISTS {keyspace_name} WITH REPLICATION = {{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }}"
session.execute(query_create_keyspace)

query_use_keyspace = f"USE {keyspace_name}"
session.execute(query_use_keyspace)

#query_create_table = f"CREATE TABLE IF NOT EXISTS {table_name} (id UUID PRIMARY KEY, Lat double, Lon double , BaseString TEXT , year INT ,month INT ,day INT, prediction INT)"
query_create_table = f"CREATE TABLE IF NOT EXISTS {table_name} (id UUID , Lat double, Lon double , BaseString TEXT , year INT ,month INT ,day INT,prediction INT,PRIMARY KEY (prediction, id))"
session.execute(query_create_table)

rows = df.collect()
for row in rows:
    id_value = uuid.uuid4()  
    query_insert = f"INSERT INTO {table_name} (id, Lat, Lon, BaseString, year, month, day, prediction) VALUES ({id_value}, {row.Lat}, {row.Lon}, '{row.BaseString}', {row.year}, {row.month}, {row.day}, {row.prediction})"
    session.execute(query_insert)

query_select = f"SELECT * FROM {table_name}"
result = session.execute(query_select)
# /////////////////////////////////////////////////////////////////////////
for row in result:
    print(row)
    
# session.shutdown()
# cluster.shutdown()

In [ ]:
# cluster = Cluster(['localhost'])
# session = cluster.connect()

# keyspace_name = 'tam8'
# table_name = 'my_table8'

#query_use_keyspace = f"USE {keyspace_name}"
#session.execute(query_use_keyspace)

#query_select = f"SELECT * FROM {table_name}"
#result = session.execute(query_select)

#SELECT prediction, COUNT(*) AS count FROM my_table8 WHERE prediction >= 0 GROUP BY prediction
# query_one = "SELECT prediction, COUNT(*) AS count FROM my_table8 WHERE prediction >= 0 GROUP BY prediction"
query_one = "SELECT prediction, COUNT(*) AS count FROM my_table8 WHERE prediction >= 0 GROUP BY prediction ALLOW FILTERING"
result_one = session.execute(query_one)

for row in result_one:
    prediction = row.prediction
    count = row.count
    print(f"Cluster: {prediction}, Count: {count}")

In [ ]:
query_use_keyspace = f"USE {keyspace_name}"
session.execute(query_use_keyspace)

query_select_data = f"SELECT * FROM {table_name}"
result = session.execute(query_select_data)

df = pd.DataFrame(list(result))

grouped = df.groupby('prediction').size().reset_index(name='count')

print(grouped)

#session.shutdown()
#cluster.shutdown()


In [ ]:
start_date = datetime(2014, 8, 1)
end_date = start_date + timedelta(days=7)

start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")

#query = f"SELECT * FROM {table_name} WHERE day >= '{start_date_str}' AND day <= '{end_date_str}'"
#query = f"SELECT * FROM {table_name} WHERE day >= {start_date.day} AND day <= {end_date.day}"
#query = f"SELECT * FROM {table_name} WHERE day >= {start_date.day} AND day <= {end_date.day} ALLOW FILTERING"
query = f"SELECT * FROM {table_name} WHERE year = {start_date.year} AND month = {start_date.month} AND day >= {start_date.day} AND day <= {end_date.day} ALLOW FILTERING"
result = session.execute(query)

for row in result:
    print(row)

In [ ]:


start_date = datetime(2014, 8, 14)
end_date = start_date + timedelta(days=7)

start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")

#query = f"SELECT * FROM {table_name} WHERE day >= '{start_date_str}' AND day <= '{end_date_str}'"
#query = f"SELECT * FROM {table_name} WHERE day >= {start_date.day} AND day <= {end_date.day}"
#query = f"SELECT * FROM {table_name} WHERE day >= {start_date.day} AND day <= {end_date.day} ALLOW FILTERING"
query = f"SELECT * FROM {table_name} WHERE year = {start_date.year} AND month = {start_date.month} AND day >= {start_date.day} AND day <= {end_date.day} ALLOW FILTERING"
result = session.execute(query)

for row in result:
    print(row)

In [ ]:
start_date = datetime(2014,8, 1) 
end_date = datetime(2014, 8, 7)    

start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")

#query = f"SELECT * FROM {table_name} WHERE day >= '{start_date_str}' AND day <= '{end_date_str}' ALLOW FILTERING"
query = f"SELECT * FROM {table_name} WHERE year = {start_date.year} AND month = {start_date.month} AND day >= {start_date.day} AND day <= {end_date.day} ALLOW FILTERING"
result = session.execute(query)

for row in result:
    print(row)

In [ ]:
# cluster = Cluster(['localhost'])
# session = cluster.connect()

# keyspace_name = 'tam88'
# table_name = 'my_table88'

# query_use_keyspace = f"USE {keyspace_name}"
# session.execute(query_use_keyspace)


from collections import defaultdict

query = f"SELECT Lat, Lon, COUNT(id) AS activity_count FROM {keyspace_name}.{table_name} WHERE year = 2014 AND month = 8 AND day >= 2 AND day <= 8 GROUP BY Lat, Lon ALLOW FILTERING"
result = session.execute(query)

cluster_activities = defaultdict(int)

for row in result:
#     cluster_activities[(row.Lat, row.Lon)] = row.activity_count
    cluster_activities[(row[0], row[1])] = row[2]

max_activity_cluster = max(cluster_activities, key=cluster_activities.get)
activity_count = cluster_activities[max_activity_cluster]

print("Cluster with the highest activity in the week:")
print("Lat:", max_activity_cluster[0])
print("Lon:", max_activity_cluster[1])
print("Activity Count:", activity_count)

# session.shutdown()
# cluster.shutdown()


In [ ]:


# cluster = Cluster(['localhost'])
# session = cluster.connect()

# keyspace_name = 'tam88'
# table_name = 'my_table88'

query_use_keyspace = f"USE {keyspace_name}"
session.execute(query_use_keyspace)


query = f"SELECT prediction, COUNT(id) AS activity_count FROM {keyspace_name}.{table_name} WHERE year = 2014 AND month = 8 AND day >= 2 AND day <= 8 ALLOW FILTERING"
result = session.execute(query)

if result:
    cluster_activities = defaultdict(int)
    
    for row in result:
        cluster_activities[row.prediction] = row.activity_count
    
    max_activity_cluster = max(cluster_activities, key=cluster_activities.get)
    activity_count = cluster_activities[max_activity_cluster]
    
    print("Cluster with the highest activity in the week:")
    print("Cluster Prediction:", max_activity_cluster)
    print("Activity Count:", activity_count)
else:
    print("No activity found in the specified week for any cluster.")

# session.shutdown()
# cluster.shutdown()


In [ ]:
df = spark.read.csv("output.csv", header=True)  
cluster = Cluster(['localhost'])
session = cluster.connect()

keyspace_name = 'tam88'
table_name = 'my_table88'

query_create_keyspace = f"CREATE KEYSPACE IF NOT EXISTS {keyspace_name} WITH REPLICATION = {{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }}"
session.execute(query_create_keyspace)

query_use_keyspace = f"USE {keyspace_name}"
session.execute(query_use_keyspace)

query_create_table = f"CREATE TABLE IF NOT EXISTS {table_name} (id UUID , Lat double, Lon double , BaseString TEXT , year INT ,month INT ,day INT, prediction INT,PRIMARY KEY (Lat,Lon,prediction, id))"
session.execute(query_create_table)

rows = df.collect()
for row in rows:
    id_value = uuid.uuid4()  
    query_insert = f"INSERT INTO {table_name} (id, Lat, Lon, BaseString ,year,month,day,prediction) VALUES ({id_value}, {row.Lat}, {row.Lon}, '{row.BaseString}', {row.year}, {row.month},{row.day}, {row.prediction})"
    session.execute(query_insert)
    
query_select = f"SELECT * FROM {table_name}"
result = session.execute(query_select)

for row in result:
    print(row)
    
#session.shutdown()
#cluster.shutdown()

In [ ]:
# cluster = Cluster(['localhost'])
# session = cluster.connect()

# keyspace_name = 'tam88'
# table_name = 'my_table88'

# query_use_keyspace = f"USE {keyspace_name}"
# session.execute(query_use_keyspace)

# query_select = f"SELECT * FROM {table_name}"
# result = session.execute(query_select)


end_date = datetime.now().date()
start_date = end_date - timedelta(days=9)  
start_year = start_date.year
start_month = start_date.month
start_day = start_date.day

end_year = end_date.year
end_month = end_date.month
end_day = end_date.day

query = f"SELECT Lat, Lon, COUNT(id) AS traffic_count FROM {table_name} WHERE year = {start_year} AND month = {start_month} AND day >= {start_day} AND day <= {end_day} GROUP BY Lat, Lon ALLOW FILTERING"
result = session.execute(query)

for row in result:
    print(row)


In [ ]:
# cluster = Cluster(['localhost'])
# session = cluster.connect()

# keyspace_name = 'tam88'
# table_name = 'my_table88'

start_date = datetime(2014, 8, 2).date()
end_date = datetime(2014, 8, 12).date()

start_year = start_date.year
start_month = start_date.month
start_day = start_date.day

end_year = end_date.year
end_month = end_date.month
end_day = end_date.day

# query_use_keyspace = f"USE {keyspace_name}"
# session.execute(query_use_keyspace)
#query = f"SELECT Lat, Lon, COUNT(id) AS traffic_count FROM {keyspace_name}.{table_name} WHERE year = {start_year} AND month = {start_month} AND day >= {start_day} AND day <= {end_day} GROUP BY Lat, Lon ALLOW FILTERING"
query = f"SELECT Lat, Lon, COUNT(id) AS traffic_count FROM {table_name} WHERE year = {start_year} AND month = {start_month} AND day >= {start_day} AND day <= {end_day} GROUP BY Lat, Lon ALLOW FILTERING"
result = session.execute(query)

for row in result:
    print(row)


In [ ]:
# cluster = Cluster(['localhost'])
# session = cluster.connect()

# keyspace_name = 'tam88'
# table_name = 'my_table88'

specified_date = datetime(2014, 8, 31).date()
start_date = specified_date - timedelta(days=10) 

start_year = start_date.year
start_month = start_date.month
start_day = start_date.day

end_year = specified_date.year
end_month = specified_date.month
end_day = specified_date.day

query_use_keyspace = f"USE {keyspace_name}"
session.execute(query_use_keyspace)
 
query = f"SELECT Lat, Lon, COUNT(id) AS traffic_count FROM {table_name} WHERE year = {start_year} AND month = {start_month} AND day >= {start_day} AND day <= {end_day} GROUP BY Lat, Lon ALLOW FILTERING"
result = session.execute(query)

for row in result:
    print(row)
